In [8]:
import torch.nn as nn
import torch
from transformers import RobertaTokenizer, RobertaModel
import numpy as np
from scipy.spatial.distance import cdist

In [9]:
class Model(nn.Module):
    def __init__(self, encoder):
        super(Model, self).__init__()
        self.encoder = encoder

    def forward(self, code_inputs=None, nl_inputs=None):
        if code_inputs is not None:
            outputs = self.encoder(code_inputs,attention_mask=code_inputs.ne(1))[0]
            outputs = (outputs*code_inputs.ne(1)[:,:,None]).sum(1)/code_inputs.ne(1).sum(-1)[:,None]
            return torch.nn.functional.normalize(outputs, p=2, dim=1)
        else:
            outputs = self.encoder(nl_inputs,attention_mask=nl_inputs.ne(1))[0]
            outputs = (outputs*nl_inputs.ne(1)[:,:,None]).sum(1)/nl_inputs.ne(1).sum(-1)[:,None]
            return torch.nn.functional.normalize(outputs, p=2, dim=1)


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Path to your model.bin file on Google Drive
# model_path = '/content/drive/My Drive/model (7).bin'
model_path = r"C:\Users\agianolini\OneDrive - ANDES WEALTH MANAGEMENT SA\Desktop\research-assistant-main\model (7).bin"

In [11]:
class CodeSearcher:
    def __init__(self, model_path, code_snippets, device=None):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device
        self.tokenizer = RobertaTokenizer.from_pretrained('microsoft/unixcoder-base')
        self.model = self.load_model(model_path)
        self.code_snippets = code_snippets
        self.code_embeddings = self.generate_code_embeddings(code_snippets)

    def load_model(self, model_path):
        model = RobertaModel.from_pretrained('microsoft/unixcoder-base')
        model = Model(model)
        model.load_state_dict(torch.load(model_path, map_location=self.device))
        model.to(self.device)
        model.eval()
        return model

    def generate_code_embeddings(self, code_snippets):
        embeddings = []
        for snippet in code_snippets:
            inputs = self.tokenizer.encode_plus(snippet, add_special_tokens=True, max_length=256, truncation=True, padding='max_length', return_tensors='pt')
            with torch.no_grad():
                embedding = self.model(code_inputs=inputs['input_ids'].to(self.device))
            embeddings.append(embedding.cpu().numpy())
        return np.vstack(embeddings)

    def get_query_embedding(self, query):
        inputs = self.tokenizer.encode_plus(query, add_special_tokens=True, max_length=128, truncation=True, padding='max_length', return_tensors='pt')
        with torch.no_grad():
            embedding = self.model(code_inputs=inputs['input_ids'].to(self.device))
        return embedding.cpu().numpy()

    def get_similarity_search(self, query, k):
        query_embedding = self.get_query_embedding(query)
        similarities = 1 - cdist(query_embedding, self.code_embeddings, 'cosine').flatten()

        # Get top-k indices
        top_k_indices = np.argsort(similarities)[-k:]

        # Create a dictionary for the top k results
        results = {}
        for index in reversed(top_k_indices):
            snippet_info = {
                'index': index,
                'snippet': self.code_snippets[index],
                'similarity': similarities[index]
            }
            results[f'top_{k}'] = snippet_info
            k -= 1

        return results




In [12]:
# Example usage
code_snippets = ["print('Hello, world!')", "my ass is like the sky!", "for i in range(10): print(i)"]  # Your collection of code snippets
searcher = CodeSearcher(model_path, code_snippets)

In [15]:
query = "print hello world"
k = 3
t = searcher.get_similarity_search(query, k)

print("Top K similar items: \n")
t

Top K similar items: 



{'top_3': {'index': 0,
  'snippet': "print('Hello, world!')",
  'similarity': 0.5174323756887338},
 'top_2': {'index': 1,
  'snippet': 'my ass is like the sky!',
  'similarity': 0.3868753964239914},
 'top_1': {'index': 2,
  'snippet': 'for i in range(10): print(i)',
  'similarity': 0.25039381632295665}}